In [85]:
import pandas as pd
import yfinance as yf
import numpy as np

In [86]:
symbol = 'TCS.NS'
df = yf.download(symbol, start = '2020-01-01', end = '2024-12-31')['Close']
df

[*********************100%***********************]  1 of 1 completed


Ticker,TCS.NS
Date,
2020-01-01,1944.476929
2020-01-02,1935.550659
2020-01-03,1974.124756
2020-01-06,1973.945312
2020-01-07,1978.789185
...,...
2024-12-23,4093.052979
2024-12-24,4113.920410
2024-12-26,4103.683594


In [87]:
from arch import arch_model

In [88]:
df['returns'] = df['TCS.NS'].pct_change() * 100
returns = df['returns'].dropna()

In [89]:
model = arch_model(returns, vol = 'ARCH', p=1)
results = model.fit(disp = 'off')
results.summary

<bound method ARCHModelResult.summary of                       Constant Mean - ARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -2268.47
Distribution:                  Normal   AIC:                           4542.93
Method:            Maximum Likelihood   BIC:                           4558.29
                                        No. Observations:                 1236
Date:                Mon, Jun 02 2025   Df Residuals:                     1235
Time:                        12:09:25   Df Model:                            1
                                 Mean Model                                
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.068

In [90]:
# Analysis of ARCH Model
# mu---> 0.0686 : The model estimates that the average daily return is.
# omega---> 2.0213 : The long level base level of variance
# alpha[1]---> 0.1538 : How much yesterday's squared shock impacts today's variance

In [91]:
forecast = results.forecast(horizon = 5)
predicted_variance = forecast.variance
predicted_volatility = predicted_variance ** 0.5
predicted_volatility

,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-30,1.424208,1.527529,1.542809,1.545146,1.545505


In [92]:
predicted_volatility = [1.555161, 1.547184, 1.545953, 1.545764, 1.545735]
predicted_avg = sum(predicted_volatility)/len(predicted_volatility)
predicted_avg

1.5479593999999999

In [93]:
real_df = yf.download('TCS.NS', start = '2024-12-31', end = '2025-01-08')['Close']
real_df['returns'] = real_df.pct_change() * 100
real_df = real_df.dropna()
real_df

[*********************100%***********************]  1 of 1 completed


Ticker,TCS.NS,returns
Date,,
2025-01-01,4047.922852,0.431040
2025-01-02,4110.229492,1.539225
2025-01-03,4035.569336,-1.816447
2025-01-06,4030.746338,-0.119512
2025-01-07,3965.093018,-1.628813


In [94]:
realized_vol = real_df['returns'].std()
realized_vol

1.41538993253752

In [95]:
print("ARCH Predicted Volatility:", predicted_avg)
print("ARCH Actual Volatility:", realized_vol)

ARCH Predicted Volatility: 1.5479593999999999
ARCH Actual Volatility: 1.41538993253752


In [103]:
model_1 = arch_model(returns, vol = 'GARCH', p=1, q=1)
results_1 = model.fit(disp = 'off')
results_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2200.91
Distribution:                  Normal   AIC:                           4409.81
Method:            Maximum Likelihood   BIC:                           4430.29
                                        No. Observations:                 1236
Date:                Mon, Jun 02 2025   Df Residuals:                     1235
Time:                        12:10:31   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0697  3.895e-02      1.789  7.364e-02 [-6.666e-03,  0.146]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.1458  6.619e-02      2.202  2.764e-02 [1.604e-02,  0.276]
alpha[1]       0.0779  3.255e-02      2.394  1.667e-02 [1.413e-02,  0.142]
beta[1]        0.8562  5.346e-02     16.015  1.009e-57   [  0.751,  0.961]
==========================================================================

Covariance estimator: robust
"""

In [104]:
# Analysis of GARCH Model
# mu---> 0.0697 : The model estimates that the average daily return is.
# omega---> 0.1458 : The long level base level of variance
# alpha[1]---> 0.0779 : How much yesterday's squared shock impacts today's variance
# beta[1]---> 0.8562 : How much yesterday's variance impact today's variance

In [105]:
forecast_1 = results_1.forecast(horizon = 5)
predicted_variance_1 = forecast_1.variance
predicted_volatility_1 = predicted_variance_1 ** 0.5
predicted_volatility_1

,h.1,h.2,h.3,h.4,h.5
Date,,,,,
2024-12-30,1.271822,1.287125,1.301257,1.314321,1.326407


In [106]:
predicted_volatility_1 = [1.271822,1.287125,1.301257,1.314321,1.326407]
predicted_avg_1 = sum(predicted_volatility_1)/len(predicted_volatility_1)
predicted_avg_1

1.3001864

In [107]:
realized_vol

1.41538993253752

In [108]:
print("GARCH Predicted Volatility:", predicted_avg_1)
print("GARCH Actual Volatility:", realized_vol)

GARCH Predicted Volatility: 1.3001864
GARCH Actual Volatility: 1.41538993253752


In [110]:
lamda = 0.94

In [111]:
ewma_var = []
var_t = df['returns'].var()

In [112]:
for ret in df['returns']:
    variance_tplus1 = lamda * var_t + (1-lamda) * (ret**2)
    ewma_var.append(variance_tplus1)

In [113]:
df['ewma_vol'] = np.sqrt(ewma_var)

In [114]:
latest_daily_vol = df['ewma_vol'].iloc[-1]
latest_daily_vol

1.4945586687558257

In [124]:
real_ewma = yf.download('TCS.NS', start = '2025-01-01', end = '2025-01-03')
real_ewma['returns'] = real_ewma['Close'].pct_change()*100
real_ewma = real_ewma.dropna()
real_vol = real_ewma['returns']
real_vol

[*********************100%***********************]  1 of 1 completed


Date
2025-01-02    1.539225
Name: returns, dtype: float64

In [128]:
print("EWMA Model:", latest_daily_vol)
print("EWMA Actual:", real_vol)

EWMA Model: 1.4945586687558257
EWMA Actual: Date
2025-01-02    1.539225
Name: returns, dtype: float64
